<a href="https://colab.research.google.com/github/JMCastle98/Programacion-con-R-Santander/blob/master/M3_Postwork7/Postwork_07_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postwork 07: Transformación, filtración y ordenamiento de datos 
Antes de comenzar en caso de no tener nuestro confiable archivo csv a la mano, o bien debido a que la API está en constante actualización podemos recurrir a una versión mejorada de la función para peticiones de los **enfrentamientos**:

In [3]:
import requests #Importamos los paquete necesarios
import pandas as pd 
import math as mt
 
def PandaScore_get_matches(juego): #Definimos nuestra función
    if isinstance(juego, str): #Nos aseguramos que los argumentos sean de tipo str
        informacion = 'matches'
        registros = 100 #Establecemos el número de registros por página
        token = 'V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns' #Token de acceso
        url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&token={token}" #URL con los parámetros deseados
        response = requests.get(url) #Realizamos la petición
        if response.status_code == 200: #Si todo salió bien continuamos
            Iteraciones = mt.ceil(float(response.headers['X-Total'])/registros) #Calculamos el número de páginas en base a todos los registros que hay en la API
            #Al dividir el número total de registros entre los registros solicitados obtenemos un aproximado, para asegurar que tomaremos todos, redondeamos al entero más alto
            for i in range(1,Iteraciones+1): #Ciclo for para obtener todos los registros de cada página
                url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&page[number]={i}&token=V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns"
                #URL el parámetro de página deseada
                response = requests.get(url) #Realizamos la petición correspondiente
                data = response.json() #Transformamos la respuesta 
                if i == 1: #En la primer iteración se crea un Dataframe
                    df_matches = pd.DataFrame.from_dict(data)
                else: #En el resto de iteraciones se van añadiendo los nuevos registros
                    df_matches = df_matches.append(pd.DataFrame.from_dict(data),ignore_index=True) 
            
            ########### Limpieza de datos
            campos_no_deseados = ['videogame', 'slug', 'tournament', 'league', 'serie', 'streams', 'modified_at', 'games',
                      'game_advantage', 'scheduled_at', 'original_scheduled_at', 'detailed_stats','live',
                      'match_type', 'videogame_version', 'live_embed_url', 'id'] 

            df_matches_temp = df_matches.drop(columns=campos_no_deseados) #Limpiamos campos que no nos son de utilidad
            df_matches_finished = df_matches_temp[ df_matches_temp['status'] == 'finished' ] #Dejamos solo los enfrentamientos que han terminado
            df_matches_finished_clean = df_matches_finished.dropna(subset=['begin_at', 'end_at']) #Eliminamos algunos valores nulos de las fechas
            df_matches_ordenado = df_matches_finished_clean[['name','opponents','results','forfeit','draw', #Reorganizamos los campos
                                                      'number_of_games','winner','winner_id','begin_at','end_at', 
                                                      'rescheduled','official_stream_url','league_id', 'serie_id', 'tournament_id']]
            df_matches_ordenado = df_matches_ordenado.reset_index(drop=True) #Reseteamos el índice
            nombres_nuevos = {'name' : 'match_name',
                             'begin_at' : 'begin_date',
                             'end_at' : 'end_date',
                             'official_stream_url' : 'official_stream_channel_url'}                       
            df_matches_ordenado.rename(columns=nombres_nuevos, inplace=True) #Renombramos columnas                                      
            return df_matches_ordenado # Finalmente se devuelve el Dataframe con todos los registros limpios y ordenados
        else: #Si la petición no se ejecutó correctamente mandamos un mensaje con el error devuelto
            return 'Error de petición: '+str(response.status_code)  
    else: #Si los argumentos no son de tipo str regresamos un mensaje
        return 'Por favor coloque un argumento válido en formato string.'

Le indicamos que deseamos los enfrentamientos del juego League of Legends:


In [4]:
lol_matches = PandaScore_get_matches('lol')

In [10]:
lol_matches.head()

,match_name,opponents,results,forfeit,draw,number_of_games,winner,winner_id,begin_date,end_date,rescheduled,official_stream_channel_url,league_id,serie_id,tournament_id
0,ALF vs BJD,"[{'opponent': {'acronym': 'ALF', 'id': 124408,...","[{'score': 0, 'team_id': 124408}, {'score': 1,...",False,False,1,"{'acronym': 'BJD', 'id': 126861, 'image_url': ...",126861.0,2021-03-06T09:54:42Z,2021-03-06T10:28:54Z,False,https://www.twitch.tv/lolpacific,4288,3337,5483
1,Lower Bracket Round 2 Match 1: CGG vs GGE,"[{'opponent': {'acronym': 'CGG', 'id': 127494,...","[{'score': 1, 'team_id': 127494}, {'score': 3,...",False,False,5,"{'acronym': 'GGE', 'id': 128405, 'image_url': ...",128405.0,2021-03-08T18:07:48Z,2021-03-08T21:37:26Z,False,https://www.twitch.tv/pg_esports,4405,3291,5579
2,Round 1 match 1: KVM vs ION,"[{'opponent': {'acronym': 'KVM', 'id': 127475,...","[{'score': 3, 'team_id': 127475}, {'score': 0,...",False,False,5,"{'acronym': 'KVM', 'id': 127475, 'image_url': ...",127475.0,2021-03-08T18:04:03Z,2021-03-08T20:46:01Z,False,https://www.twitch.tv/esportsprimelol,4401,3248,5609
3,Winners bracket round 1 match 2: ESB vs SIN,"[{'opponent': {'acronym': 'ESB', 'id': 3344, '...","[{'score': 2, 'team_id': 3344}, {'score': 3, '...",False,False,5,"{'acronym': 'SIN', 'id': 127586, 'image_url': ...",127586.0,2021-03-07T17:53:43Z,2021-03-07T22:22:51Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602
4,Winners bracket round 1 match 1: IG vs VIK,"[{'opponent': {'acronym': 'IG', 'id': 127585, ...","[{'score': 3, 'team_id': 127585}, {'score': 1,...",False,False,5,"{'acronym': 'IG', 'id': 127585, 'image_url': '...",127585.0,2021-03-07T13:30:41Z,2021-03-07T17:08:57Z,False,https://www.twitch.tv/hitpointcz,4433,3314,5602


Antes de comenzar a tratar los datos, nos gustaría modificar un poco la información que mostramos aprovechando 3 campos que dejamos en forma de lista:

*   _oponents_
*   _results_
*   _winner_


